In [ ]:
#%pip install torchtoolbox
import random
import os, sys
from albumentations import *
import albumentations as A
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import torch
import seaborn as sns
import torchvision
from torchvision import transforms
from torchvision.transforms import *
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score,recall_score
from torchvision.datasets import *
import sklearn
from sklearn.model_selection import train_test_split
import cv2
import albumentations
import glob
import shutil
from torchvision.transforms import *
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch.transforms import ToTensorV2
from torch import nn
import gc
device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision import datasets, models, transforms
#from pytorchtools.pytorchtools import EarlyStopping
from torchtoolbox.transform import Cutout as ct
from torchvision.transforms import *
from PIL import Image
import matplotlib.pyplot as plt

# 해야할것
### 1. args에서 pt.file_path 수정하기 and name에 영문이름 입력
### 2. 기존에 승규님에게 받은 모델들 복붙하기 

# args pt.file_path랑 name 넣어주세요

In [ ]:
args = {
    'name' : "hoyeon",
    # 이름 수정하고 입력해주세요
    'pt.file_path' : "../input/model-rs-200crop170/model_Rs200Crop170.pt",
    #'pt.file_path' : "1)Add Data에 다운받기 2)Input에 있는 pt파일 경로 가져오기",
    'dataset_path' : "../input/campdataset/camp23310-copy",
    #train:valid = 10:2
    'valid_size' : 0.2,
    'img_size' : 256,

    #train,valid 데이터는 이미 여러가지 변환이 적용되어 있음
    #그러므로,정규화와 Resize만 공통적용
    'tr_transform' : transforms.Compose([
                transforms.Resize(256),  #여기
                transforms.CenterCrop(224), #여기
                #ct(p=1.0,scale=(0.01,0.1),value=(0,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ]),
    'val_transform' :  transforms.Compose([
                transforms.Resize(256),  #여기
                transforms.CenterCrop(224),#여기
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ]),
    'batch_size' : 32,
    'lr' : 0.001,
    'cls_num' : 20
}

In [ ]:
args["pt.file_path"]

'../input/model-rs-200crop170/model_Rs200Crop170.pt'

In [ ]:
dataset = torchvision.datasets.ImageFolder("../input/campdataset/camp23310-copy")

In [ ]:
dataset

Dataset ImageFolder
    Number of datapoints: 97987
    Root location: ../input/campdataset/camp23310-copy

In [ ]:
train_indices , valid_indices = train_test_split(np.arange(len(dataset.targets)),test_size = args["valid_size"],stratify = dataset.targets,random_state = 1234)
train_indices

array([34074, 54002, 16989, ..., 78035,  9773,  9138])

# model,복붙해야해요 

In [ ]:
class Subdata(Dataset):
    def __init__(self,dataset,indices,transforms):
        self.dataset = dataset
        self.indices = indices
        self.transform = transforms
    def __getitem__(self,idx):
        image,label = self.dataset[self.indices[idx]] #여기 중요!!.올바르게 인덱스 불러오려면 .. 제대로입력
        image = self.transform(image)
        return image,label
    def __len__(self):
        return len(self.indices)

training_dataset = Subdata(dataset,indices = train_indices,transforms = args["tr_transform"])
validation_dataset = Subdata(dataset,indices = valid_indices,transforms = args["val_transform"])
print(len(training_dataset),len(validation_dataset))

78389 19598


In [ ]:
tr_loader = torch.utils.data.DataLoader(dataset = training_dataset,batch_size = args["batch_size"],shuffle =  True)
val_loader = torch.utils.data.DataLoader(dataset = validation_dataset,batch_size = args["batch_size"], shuffle = True)

In [ ]:
def conv_block_1(in_dim,out_dim, activation,stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=1, stride=stride),
        nn.BatchNorm2d(out_dim),
        activation,
    )
    return model


def conv_block_3(in_dim,out_dim, activation, stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=3, stride=stride, padding=1, groups=4),
        nn.BatchNorm2d(out_dim),
        activation,
    )
    return model


In [ ]:
class SEModule(nn.Module):
    
    def __init__(self, in_dim, r=4):       # SE block에 들어가는 채널수는 out_dim과 동일
        super(SEModule, self).__init__()
        self.Avgpool = nn.AdaptiveAvgPool2d(1)      # 1만 입력해야 Global으로 동작

        self.excitation = nn.Sequential(
          nn.Linear(in_dim, in_dim//r, bias=False),
          nn.ReLU(),
          nn.Linear(in_dim//r, in_dim, bias=False),
          nn.Sigmoid()
        )


    
    def forward(self, x):      
        batch, channel, _, _, = x.size()
        se = self.Avgpool(x).view(batch, channel)
        se = self.excitation(se).view(batch, channel, 1, 1)
        return x * se.expand_as(x)





In [ ]:
class BottleNeck(nn.Module):
    def __init__(self,in_dim,mid_dim,out_dim,down=False):
        super(BottleNeck,self).__init__()
        self.down=down
        self.relu=nn.ReLU()
        self.se = SEModule(out_dim)


        if self.down:
            self.layer = nn.Sequential(
              conv_block_1(in_dim,mid_dim,nn.ReLU(),stride=2),
              conv_block_3(mid_dim,mid_dim,nn.ReLU(),stride=1),
              conv_block_1(mid_dim,out_dim,nn.Identity(),stride=1),
            )
            
            self.downsample = nn.Conv2d(in_dim,out_dim,kernel_size=1,stride=2)
            
        else:
            self.layer = nn.Sequential(
                conv_block_1(in_dim,mid_dim,nn.ReLU(),stride=1),
                conv_block_3(mid_dim,mid_dim,nn.ReLU(),stride=1),
                conv_block_1(mid_dim,out_dim,nn.Identity(),stride=1),
            )
            
        self.dim_equalizer = nn.Conv2d(in_dim,out_dim,kernel_size=1)
                  
    def forward(self,x):
        if self.down:
            downsample = self.downsample(x)
            out = self.layer(x)
            out = self.se(out)
            out = out + downsample
            out = self.relu(out)    # 그냥 nn.ReLU치면 안되고 self.relu로 만들어서 해야 함
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_equalizer(x)
            
            out = self.se(out)
            out = out + x
            out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):

    def __init__(self, base_dim, num_classes=args["cls_num"]):
        super(ResNet, self).__init__()
        self.activation = nn.ReLU()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            BottleNeck(base_dim, base_dim*2, base_dim*4, down=False),
        )   
        self.layer_3 = nn.Sequential(
            BottleNeck(base_dim*4,base_dim*8,base_dim*16, down=True),
        )

        self.avgpool = nn.AvgPool2d(1,1) 
        self.flatten = nn.Flatten()
        #self.fc_layer = nn.Linear(base_dim*16*9*9,num_classes)
        self.fc_layer = nn.Linear(12544,num_classes)

        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_uniform_(module.weight.data, mode='fan_in', nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.zero_()
                
        
    def forward(self, x):
        out = self.layer_1(x)
        #print("0",out.shape)
        out = self.layer_2(out)
        #print("1",out.shape)
        out = self.layer_3(out)
        #print("2",out.shape)
       # out = self.layer_4(out)
        #print("3",out.shape)
       # out = self.layer_5(out)
        #print("4",out.shape)
        out = self.avgpool(out)
        #print("avg 5:",out.shape)
        out = self.flatten(out)
        #print('fltn 6:',out.shape)
        out = self.fc_layer(out)
        #print('fc 7:',out.shape)
        
        return out

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
gc.collect()
torch.cuda.empty_cache()



#model = ResNet(base_dim=16).to(device) #pytorch resnet
#model = torch.load("../input/modelresize60centercrop50/model.pt")
#torch.save(model, '/kaggle/working' + '/model_Rs60Crop50.pt')


#torch.save(model.state_dict(), '/kaggle/working' + '/model_state_dictRs100Crop80.pt')  # 모델 객체의 state_dict 저장


model = torch.load(args["pt.file_path"])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"])
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

cuda


In [ ]:
model.fc_layer = torch.nn.Linear(200704,20)
model.to("cuda") #model.to(device)

ResNet(
  (activation): ReLU()
  (layer_1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer_2): Sequential(
    (0): BottleNeck(
      (relu): ReLU()
      (se): SEModule(
        (Avgpool): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (0): Linear(in_features=64, out_features=16, bias=False)
          (1): ReLU()
          (2): Linear(in_features=16, out_features=64, bias=False)
          (3): Sigmoid()
        )
      )
      (layer): Sequential(
        (0): Sequential(
          (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4)
          (1): BatchNorm2d(32, e

In [ ]:
str('hi')

'hi'

In [ ]:
max_val_acc = 0
patience = 0
earlystop_threshold = 3
epoches = 1000000


train_epoch_loss_history = []
train_epoch_acc_history = []
valid_epoch_loss_history = []
valid_epoch_acc_history = []
for epoch in range(epoches):
    
    train_epoch_loss = 0.0
    train_epoch_correct = 0.0
    
    valid_epoch_loss = 0.0
    valid_epoch_correct = 0.0
    
    
    model.train()
    batch_idx = 0
    for X_train,y_train in tr_loader:
        X_train = X_train.to(device);y_train = y_train.to(device)
        #print("tr",X_train.shape)
        #1yhat
        yhat_tr = model(X_train)
        #2loss
        loss = loss_fn(yhat_tr,y_train)
        #3derivative
        loss.backward()
        #4 update & clean
        optimizer.step()
        optimizer.zero_grad()
    
        #ck code
        model.eval()
        #loss_record
        train_epoch_loss += loss.item()
        #correct_record
        pred_y = yhat_tr.argmax(dim=1,keepdim=True)
        correct = pred_y.eq(y_train.view_as(pred_y)).sum().item()
        train_epoch_correct += correct
        #print("train_cor",train_epoch_correct)
        #print(batch_idx)
        if batch_idx % 500 == 0:
            print(f'epoch : {epoch+1} batch : {batch_idx}th',correct)
        batch_idx += 1
    
    with torch.no_grad():
        for X_valid,y_valid in val_loader:
            X_valid = X_valid.to(device)
            y_valid = y_valid.to(device)
            #print(X_valid.shape,y_valid.shape)
            #1yhat
            yhat_val = model(X_valid)
            #2loss
            loss_val = loss_fn(yhat_val,y_valid)
            
            #ckcode
            #loss_record
            valid_epoch_loss += loss_val.item()
            #correct_record
            pred_y = yhat_val.argmax(dim=1,keepdim=True)
            correct = pred_y.eq(y_valid.view_as(pred_y)).sum().item()
            valid_epoch_correct += correct
            #print(correct)
    ep_tr_loss = train_epoch_loss / len(training_dataset) ; ep_tr_acc = train_epoch_correct / len(training_dataset)
    train_epoch_loss_history.append(ep_tr_loss) ; train_epoch_acc_history.append(ep_tr_acc)
    
    ep_val_loss = valid_epoch_loss / len(validation_dataset) ; ep_val_acc = valid_epoch_correct / len(validation_dataset)
    valid_epoch_loss_history.append(ep_val_loss) ; valid_epoch_acc_history.append(ep_tr_acc)
    
    if ep_val_acc > max_val_acc:
        max_val_acc = ep_val_acc
        patience = 0
    else:
        patience +=1
    
    print(f'epoch : {epoch+1} // patience : {patience}=======================================================================')
    print(f'epoch_train_loss : {ep_tr_loss} epoch_train_accuracy : {ep_tr_acc}')
    print(f'epoch_valid_loss : {valid_epoch_loss / len(validation_dataset)} epoch_valid_accuracy : {valid_epoch_correct / len(validation_dataset)}\n')
    if epoch % 5 == 0:
        path = '/kaggle/working' + '/model_final_'+str(args["name"])+"_ep" + str(epoch) +".pt"
        path2 = '/kaggle/working' + '/model_final_'+str(args["name"])+"_ep" + str(epoch) +"inform.pt"

        torch.save(model, path)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, path2)
    if patience == earlystop_threshold:
        torch.save(model, '/kaggle/working/' + 'last.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,}, path)
        break

epoch : 1 batch : 0th 17


KeyboardInterrupt: 